# SBM assignment - Data Cleaning and Inspection

### Importing packages

In [103]:
#all required imports 

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime
import numpy as np
import os
from numpy import cov

### Loading dataset

In [104]:
#load the dataset
google_last_df = pd.read_csv("C:/Users/s149077/source/repos/jasperarendsen/Strategy-and-Business-Models-JADS-Group-2/Data/google_data_last.csv",encoding='cp1252')

In [124]:
#reset dataset
google_df = google_last_df.copy()


KeyboardInterrupt



In [106]:
#check column names
google_df.columns

Index(['Unnamed: 0', 'my_app_id', 'date_published', 'privacy_policy',
       'rating_app', 'nb_rating', 'num_downloads', 'content_rating_app',
       'developer', 'categ_app', 'in_app', 'has_ads', 'price_gplay',
       'operating_system', 'software_version', 'interactive_element',
       'in_app_product', 'developer_name', 'nb_screenshots', 'description',
       'whats_new', 'email_to', 'developer_info', 'similar_apps_top15',
       'visit_website', 'more_from_developer', 'family_library',
       'permissions'],
      dtype='object')

In [107]:
#determine number off applications and check for duplicates
print(len(set(google_df)))
print(len(google_df))

28
845471


In [125]:
#Filter variables

#run the line below if you start with the full dataset
google_df = google_df[['my_app_id','rating_app','nb_rating', 'num_downloads', 'price_gplay', 'has_ads', 'in_app', 'categ_app', 'date_published', 'software_version', 'content_rating_app', 'more_from_developer', 'nb_screenshots', 'permissions', 'operating_system']].copy()

### Cleaning the data

In [118]:
#looking at the NA's in rating and number of ratings
google_df[google_df.rating_app.isna()][['rating_app']].describe()
# Here we conclude that in most cases the missing values in rating_app & nb_rating are caused by no ratings being present so for RQ1 we can safely discard the missing values.
#Now we can look at the missing values in number of downloads.

#looking at number of downloads
google_df[google_df.num_downloads.isna()]
google_df[google_df.num_downloads.isna()][['nb_rating']].describe()
#We can see that a missing value in number of downloads often pairs with a missing value in rating, number of ratings and price. The datapoints with a missing number of downloads were either never downloaded, 
#or are a result of an error whilst gathering the data. We can never be sure about the first case, so we cannot convert them all to 0. Therefore we assume they are errors and delete these datapoints.

,nb_rating
count,0
unique,0
top,NaN
freq,NaN


In [119]:
#comparing the missing values of nb_rating and rating_app
nr_missing = len(google_df[google_df.nb_rating.isna()]['nb_rating'])
corr1 = float(len(google_df[google_df.nb_rating.isna()]['nb_rating'])/len(set(google_df[google_df.nb_rating.isna()]['my_app_id'] + google_df[google_df.rating_app.isna()]['my_app_id'])))
corr2 = float(len(google_df[google_df.nb_rating.isna()]['rating_app'])/len(set(google_df[google_df.nb_rating.isna()]['my_app_id'] + google_df[google_df.rating_app.isna()]['my_app_id'])))

print(nr_missing)
print(corr1, corr2)
#value of 1.0 means that the cases containing a missing value are exactle the same for both variables 

69778
1.0 1.0


In [126]:
#create year and month published
google_df['year_published'] = google_df.date_published.apply(lambda x: str(x)[-4:])
google_df['year_published'] = google_df['year_published'].replace({'span': 2012}, regex=True)
google_df['year_published'] = google_df['year_published'].astype(float)
google_df = google_df.drop('date_published', axis=1)
google_df['age'] = google_df.year_published.apply(lambda x: 2018 - x)

#content rating of app
google_df['content_rating_app'] = google_df.content_rating_app.apply(lambda x: str(x)[:7].replace(' ', '_'))
google_df.content_rating_app = google_df.content_rating_app.replace('nan', 'nan_content', regex = True)
OH_content = pd.get_dummies(google_df['content_rating_app'])
google_df = google_df.drop('content_rating_app', axis=1)
google_df = google_df.join(OH_content)
google_df = google_df.rename(columns = {'PEGI_7_':'PEGI_7'})

#versions
google_df['version'] = google_df.software_version.apply(lambda x: str(x)[:1])
google_df = google_df.drop('software_version', axis=1)

#minimal app operating system
google_df['min_operating_system'] = google_df.operating_system.apply(lambda x: 'version_' + str(x)[:3])
google_df.min_operating_system = google_df.min_operating_system.replace('nan', 'nan_os', regex = True)
OH_OS = pd.get_dummies(google_df['min_operating_system'])
google_df = google_df.drop('min_operating_system', axis=1)
google_df = google_df.join(OH_OS)

#create number of permissions
google_df['nb_permissions'] = google_df.permissions.apply(lambda x: 0 if type(x) == float else x.count(',') + 1)

In [ ]:
#number of apps per developer
google_df['nb_apps_developer'] = google_df.more_from_developer.apply(lambda x: 1 if type(x) == float else x.count(',') + 2)

#clean num_download
google_df.num_downloads = google_df.num_downloads.str.replace('+','')
google_df.num_downloads = google_df.num_downloads.str.replace('>','')
google_df.num_downloads = google_df.num_downloads.str.replace(' ','')
google_df.num_downloads = google_df.num_downloads.str.replace(',','')
google_df.num_downloads = google_df.num_downloads.astype('float')

#add a new column with the nr of downloads as a catagory 
category = pd.cut(google_df.num_downloads, bins=[0,99,999,9999,99999,999999, 99999999999], labels=['0 - 99','100 - 999','1000 - 9999','10000 - 99999', '100000 - 999999', '1000000 +'])
google_df.insert(8,'num_downloads_cat', category)

#Make new column with free vs paid boolean
zero_values = ['0', '0 USD', '0 SEK', '0 GBP', '0 MXN', '0 HKD', '0 KRW', '0 CZK', '0 CAD', '0 AED', '0 DKK', '0 IDR', '0 TWD', '0 INR', '0 CLP', '0 SAR', '0 ILS', '0 RUB', '0 AUD', '0 PLN','0 CHF', '0 MYR', '0 TRY', '0 BRL', '0 BGN', '0 UAH', '0 JPY', '0 EUR'] 
for value in zero_values:
    google_df['price_gplay'] = google_df['price_gplay'].replace([value],0)
google_df['free_app'] = google_df['price_gplay'] ==  0

'''
#Make a variable that has all possible catagories resulting from combinations of the booleans: free_app, has_ads and in_app
category = [     'free_noads_noinapp' if     google_df.loc[row].free_app and not google_df.loc[row].has_ads and not google_df.loc[row].in_app
            else 'free_noads_inapp'   if     google_df.loc[row].free_app and not google_df.loc[row].has_ads and     google_df.loc[row].in_app
            else 'free_ads_noinapp'   if     google_df.loc[row].free_app and     google_df.loc[row].has_ads and not google_df.loc[row].in_app
            else 'free_ads_inapp'     if     google_df.loc[row].free_app and     google_df.loc[row].has_ads and     google_df.loc[row].in_app
            else 'paid_noads_noinapp' if not google_df.loc[row].free_app and not google_df.loc[row].has_ads and not google_df.loc[row].in_app 
            else 'paid_noads_inapp'   if not google_df.loc[row].free_app and not google_df.loc[row].has_ads and     google_df.loc[row].in_app
            else 'paid_ads_noinapp'   if not google_df.loc[row].free_app and     google_df.loc[row].has_ads and not google_df.loc[row].in_app
            else 'paid_ads_inapp'     for row in google_df.index]
google_df['category'] = category
'''

#clean rating variable
google_df.nb_rating = google_df.nb_rating.apply(lambda x: str(x).replace(',',''))
google_df.nb_rating = google_df.nb_rating.astype('float')


#turn catagorical variables into one-hot variables
OH_categ_app = pd.get_dummies(google_df['categ_app'])
google_df = google_df.drop('categ_app', axis=1)
google_df = google_df.join(OH_categ_app)

In [112]:
google_df = google_df.rename(columns={'PEGI 12': 'PEGI_12',
                                     'version_1.0':'version_1_0',
                                     'version_1.1':'version_1_1',
                                     'version_1.5':'version_1_5',
                                     'version_1.6':'version_1_6',
                                     'version_2.0':'version_2_0',
                                     'version_2.1':'version_2_1',
                                     'version_2.2':'version_2_2',
                                     'version_2.3':'version_2_3',
                                     'version_3.0':'version_3_0',
                                     'version_3.1':'version_3_1',
                                     'version_3.2':'version_3_2',
                                     'version_4.0':'version_4_0',
                                     'version_4.1':'version_4_1',
                                     'version_4.2':'version_4_2',
                                     'version_4.3':'version_4_3',
                                     'version_4.4':'version_4_4',
                                     'version_5.0':'version_5_0',
                                     'version_5.1':'version_5_1',
                                     'version_6.0':'version_6_0',
                                     'version_7.0':'version_7_0',
                                     'version_7.1':'version_7_1',
                                     'version_8.0':'version_8_0',
                                      'Action & Adventure':'ActionANDAdventure',
                                      'Art & Design':'ArtANDDesign',
                                      'Auto & Vehicles':'AutoANDVehicles',
                                      'Books & Reference':'BooksANDReference',
                                      'Brain Games':'BrainGames',
                                      'Food & Drink':'FoodANDDrink',
                                      'Health & Fitness':'HealthANDFitness',
                                      'House & Home':'HouseANDHome',
                                      'Libraries & Demo':'LibrariesANDDemo',
                                      'Maps & Navigation':'MapsANDNavigation',
                                      'Music & Audio':'MusicANDAudio',
                                      'Music & Video':'MusicANDVideo',
                                      'News & Magazines':'NewsANDMagazines',
                                      'Pretend Play':'PretendPlay',
                                      'Role Playing':'RolePlaying',
                                      'Travel & Local':'TravelANDLocal',
                                      'Video Players & Editors':'VideoPlayersANDEditors',
                                     })

In [ ]:
#create interaction effects
google_df['ads&purchases'] = google_df.has_ads & google_df.in_app
empty_list = []
for idx in range(len(google_df.free_app)):
    if google_df.free_app[idx] & google_df.has_ads[idx]:
        empty_list.append('free_and_ads')
    elif google_df.free_app[idx] and not google_df.has_ads[idx]:
        empty_list.append('free_no_ads')
    elif google_df.has_ads[idx] and not google_df.free_app[idx]:
        empty_list.append('paid_and_ads')
    elif not (google_df.free_app[idx] & google_df.has_ads[idx]):
        empty_list.append('paid_no_ads')

google_df['freeXads'] = empty_list

empty_list_1 = []
for idx in range(len(google_df.free_app)):
    if google_df.free_app[idx] & google_df.in_app[idx]:
        empty_list_1.append('free_and_purchases')
    elif google_df.free_app[idx] and not google_df.in_app[idx]:
        empty_list_1.append('free_no_purchases')
    elif google_df.in_app[idx] and not google_df.free_app[idx]:
        empty_list_1.append('paid_and_purchases')
    elif not (google_df.free_app[idx] & google_df.in_app[idx]):
        empty_list_1.append('paid_no_purchases')

google_df['freeXpurchases'] = empty_list_1

empty_list_2 = []
for idx in range(len(google_df.free_app)):
    if google_df.free_app[idx] & google_df['ads&purchases'][idx]:
        empty_list_2.append('free_and_purchases_and_ads')
    elif google_df.free_app[idx] and not google_df['ads&purchases'][idx]:
        empty_list_2.append('free_no_purchases_no_ads')
    elif google_df['ads&purchases'][idx] and not google_df.free_app[idx]:
        empty_list_2.append('paid_and_purchases_and_ads')
    elif not (google_df.free_app[idx] & google_df['ads&purchases'][idx]):
        empty_list_2.append('paid_no_purchases_no_ads')

google_df['freeXpurchasesANDads'] = empty_list_2

#google_df["freeXads"] = google_df["freeXads"].cat.codes
#google_df["freeXpurchases"] = google_df["freeXpurchases"].cat.codes
#google_df["freeXpurchasesANDads"] = google_df["freeXpurchasesANDads"].cat.codes

### Save the file

In [114]:
#include missing values
google_df.to_csv("C:/Users/s149077/source/repos/jasperarendsen/Strategy-and-Business-Models-JADS-Group-2/Data/cleaned_withNA.csv",index=False)

In [115]:
#do not include missing values
google_df = google_df.dropna(subset=['rating_app']) 
google_df.to_csv("C:/Users/s149077/source/repos/jasperarendsen/Strategy-and-Business-Models-JADS-Group-2/Data/cleaned_withoutNA.csv",index=False)